In [1]:
import numpy as np

In [2]:
import sys
sys.path.insert(0, '/home/adrien/MVA/KERNEL/DataChallenge-KernelMethods/Code/Alex/')

from utils import *
from svm import *

# --Data--

In [3]:
folder = '/home/adrien/MVA/KERNEL/DataChallenge-KernelMethods/Data/'

In [4]:
dataset = 0

In [5]:
data = read_data(folder, dataset=dataset)

In [7]:
print(data.keys())

dict_keys(['Ytr', 'Xtr_mat50', 'Xtr', 'Xte_mat50', 'Xte'])


# --Substring kernel--

In [6]:
def substring_kernel(x, y, gamma, k):
    """
    x, y: input strings
    gamma: real value
    k: length of substring
    
    compute substring kernel distance between x and y
    """
    
    N = len(x)
    M = len(y)
    grid_B = np.zeros((M, N, k+1))
    grid_K = np.zeros((M, N, k+1))
    
    for p in range(1, k+1):
        for i in range(M):
            for j in range(N):
                
                if min(i+1, j+1) < p:
                    continue
                
                sum_B = 0
                sum_K = 0
                
                if p == 1 and j == 0:
                    
                    for n, a in enumerate(y[0:i+1]):
                        
                        if a == x[0]:
                            
                            sum_B += gamma ** (i+1-n)
                            sum_K += gamma ** 2
                    
                    grid_B[i, 0, 1] = gamma * sum_B
                    grid_K[i, 0, 1] = sum_K
                 
                else:
                    
                    for n in range(1, i+1):
                        
                        if y[n] == x[j]:
                            
                            sum_B += grid_B[n-1, j-1, p-1] * (gamma ** (i-n+2))
                            sum_K += grid_B[n-1, j-1, p-1]
                    
                    grid_B[i, j, p] = gamma * grid_B[i, j-1, p] + sum_B
                    grid_K[i, j, p] = gamma * grid_K[i, j-1, p] + (gamma ** 2) * sum_K
    
    return grid_K[M-1, N-1, k]

In [116]:
substring_kernel('cat', 'car', 2, 2)

32.0

# --Main--

parameters

In [6]:
test_size = 0.25
tau = 1e-6
k = 4
gamma = 0.5

data

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X = data['Xtr']
Y = data['Ytr']

In [9]:
X_train, X_test, Y_train, Y_test = (train_test_split(X, Y, test_size=test_size))

model - train

In [10]:
from dev import *

In [11]:
Kernel = Kernel_Substring(k, gamma)
svm_dual = kernel_SVM(tau, X_train, Y_train, Kernel, dual=True)

Computing Spectrum Kernel Matrix:   0%|          | 1/1500 [16:03<401:04:30, 963.22s/it]

KeyboardInterrupt: 

model - test

In [ ]:
x_sol, alpha, acc_train = svm_dual.svm_solver(solver='cvxopt')
acc_test = svm_dual.compute_accuracy(X_test, Y_test, alpha)
print('acc_train', acc_train)
print('acc_test', acc_test)
print('DUAL OPTIMIZATION FINISHED')

In [16]:
len(X_train[0])

100